# Working Notebook
A notebook to work and test different ideas and thoughts in the building of forecast automation. 

In [168]:
# Libraries
import pandas as pd
import numpy as np
import yaml
import os
import pyodbc
import glob
import copy
import traceback
import warnings
warnings.filterwarnings('ignore')
from data_prep_functions import *

In [95]:
households_file = download_and_concat_Tdrive_files('36R', 'Household_Files')
households_file

,hhid,household_serial_no,taz,mgra,hinccat1,hinc,hworkers,veh,persons,hht,bldgsz,unittype,version,poverty,year
0,1,0,3458,47,5,294757,1,0,2,1,2,0,0,22.364,2016
1,2,0,3407,10,5,210480,1,0,2,1,2,0,0,15.970,2016
2,3,0,3458,16,5,236389,0,0,2,3,2,0,0,15.728,2016
3,4,0,3379,27,4,121977,2,0,2,2,2,0,0,8.353,2016
4,5,0,3358,3,4,104512,2,0,2,7,2,0,0,7.157,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455750,1455751,0,3144,6055,1,14000,0,0,1,0,9,1,0,1.234,2050
1455751,1455752,0,3144,6055,1,14000,0,0,1,0,9,1,0,1.234,2050
1455752,1455753,0,3144,6055,1,14000,0,0,1,0,9,1,0,1.234,2050
1455753,1455754,0,3144,6055,1,14000,1,0,1,0,9,1,0,1.234,2050


In [123]:
ds36R_mgra = pd.read_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Data\mgra_DS36R_ind_QA.csv')

In [141]:
# Households file: year, mgra, total_gq_population
df1 = households_file[['year', 'mgra', 'persons', 'unittype']]
df1 = df1[df1['unittype'] == 1]
df1 = df1[['year', 'mgra', 'persons']]
df1 = df1.groupby(['year', 'mgra']).sum()
df1 = df1.rename(columns={'persons': 'total_gq_population'})
df1 = df1.reset_index()
df1['year'] = pd.to_numeric(df1['year'])


df1

,year,mgra,total_gq_population
0,2016,1065,193
1,2016,1070,178
2,2016,1071,31
3,2016,1073,49
4,2016,1076,588
...,...,...,...
579,2050,19191,2694
580,2050,19214,566
581,2050,22593,577
582,2050,22705,1074


In [137]:
# CSV file: year, mgra, total_gq_population 
df2 = ds36R_mgra[['year', 'mgra', 'pop', 'hhp']]
df2['GQ_pop'] = df2['pop'] - df2['hhp']
df2 = df2[['year', 'mgra', 'GQ_pop']]
df2 = df2.groupby(['year', 'mgra']).sum()
df2.columns = ['total_gq_population_csv_file']
df2 = df2.reset_index()

df2

,year,mgra,total_gq_population_csv_file
0,2016,1,0
1,2016,2,0
2,2016,3,0
3,2016,4,0
4,2016,5,0
...,...,...,...
299021,2050,22998,0
299022,2050,22999,0
299023,2050,23000,0
299024,2050,23001,0


In [142]:
df1.dtypes

year                   int64
mgra                   int64
total_gq_population    int64
dtype: object

In [149]:
# Merge 

gq_comparison = df2.merge(df1, how='left', on=['year', 'mgra'])

gq_comparison

,year,mgra,total_gq_population_csv_file,total_gq_population
0,2016,1,0,NaN
1,2016,2,0,NaN
2,2016,3,0,NaN
3,2016,4,0,NaN
4,2016,5,0,NaN
...,...,...,...,...
299021,2050,22998,0,NaN
299022,2050,22999,0,NaN
299023,2050,23000,0,NaN
299024,2050,23001,0,NaN


In [152]:
gq_comparison = gq_comparison.rename(columns={'total_gq_population':'total_gq_population_households_file'})

In [153]:
gq_comparison[gq_comparison['total_gq_population_csv_file'] > 0]

,year,mgra,total_gq_population_csv_file,total_gq_population_households_file
120,2016,121,6,NaN
131,2016,132,66,NaN
154,2016,155,3,NaN
161,2016,162,2,NaN
164,2016,165,3,NaN
...,...,...,...,...
298930,2050,22907,3,NaN
298938,2050,22915,3,NaN
298949,2050,22926,9,NaN
299002,2050,22979,5,NaN


In [155]:
gq_comparison.to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Data\mgra_households_dataset_population_comparison_GQ_only_DS36R_QA.csv', index=False)

In [145]:
gq_comparison[gq_comparison['total_gq_population_csv_file'] != 0].to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Data\.csv')

,year,mgra,total_gq_population_csv_file,total_gq_population
120,2016,121,6,NaN
131,2016,132,66,NaN
154,2016,155,3,NaN
161,2016,162,2,NaN
164,2016,165,3,NaN
...,...,...,...,...
298930,2050,22907,3,NaN
298938,2050,22915,3,NaN
298949,2050,22926,9,NaN
299002,2050,22979,5,NaN


In [108]:
# Number of households 

n_o_h = households_file[['year','mgra']]
n_o_h['num_of_households'] = 1
n_o_h = n_o_h.groupby(['year', 'mgra']).sum()
n_o_h = n_o_h.reset_index()

n_o_h

,year,mgra,num_of_households
0,2016,1,18
1,2016,2,34
2,2016,3,52
3,2016,4,30
4,2016,5,28
...,...,...,...
237943,2050,22995,37
237944,2050,22996,103
237945,2050,22998,87
237946,2050,23000,126


In [114]:
# Total Household Population 
t_h_p = households_file[['year','mgra', 'persons']]
t_h_p = t_h_p.groupby(['year', 'mgra']).sum()
t_h_p = t_h_p.reset_index()
t_h_p = t_h_p.rename(columns={'persons':'total_household_pop'})

t_h_p

,year,mgra,total_household_pop
0,2016,1,41
1,2016,2,81
2,2016,3,111
3,2016,4,73
4,2016,5,63
...,...,...,...
237943,2050,22995,105
237944,2050,22996,248
237945,2050,22998,242
237946,2050,23000,323


In [115]:
# GQ Population 
gq_pop_households_file = households_file[households_file['unittype'] == 1]
gq_p = gq_pop_households_file[['year','mgra', 'persons']]
gq_p = gq_p.groupby(['year', 'mgra']).sum()
gq_p = gq_p.rename(columns={'persons':'total_GQ_pop'})
gq_p = gq_p.reset_index()

gq_p

,year,mgra,total_GQ_pop
0,2016,1065,193
1,2016,1070,178
2,2016,1071,31
3,2016,1073,49
4,2016,1076,588
...,...,...,...
579,2050,19191,2694
580,2050,19214,566
581,2050,22593,577
582,2050,22705,1074


In [120]:
output = n_o_h.merge(t_h_p, how='left', on=['year', 'mgra']).merge(gq_p, how='left', on=['year', 'mgra'])
output.to_csv(r'C:\Users\cra\Desktop\output.csv')

In [119]:
output[output['total_GQ_pop'] > 0]

,year,mgra,num_of_households,total_household_pop,total_GQ_pop
977,2016,1065,366,637,193.0
981,2016,1070,334,616,178.0
982,2016,1071,47,72,31.0
983,2016,1073,540,1457,49.0
984,2016,1076,588,588,588.0
...,...,...,...,...,...
235373,2050,19191,3452,4932,2694.0
235389,2050,19214,566,566,566.0
237688,2050,22593,577,577,577.0
237774,2050,22705,1074,1074,1074.0


In [121]:
households_file[households_file['mgra'] == 2518]

,hhid,household_serial_no,taz,mgra,hinccat1,hinc,hworkers,veh,persons,hht,bldgsz,unittype,version,poverty,year
52329,52330,0,4227,2518,1,10254,0,0,2,3,8,0,0,0.682,2016
52341,52342,0,4227,2518,3,82085,0,0,4,3,8,0,0,3.699,2016
52349,52350,0,4227,2518,5,153462,2,0,3,3,8,0,0,8.743,2016
52352,52353,0,4227,2518,3,78462,1,0,2,5,8,0,0,5.373,2016
52357,52358,0,4227,2518,1,7657,1,0,2,1,8,0,0,0.524,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358916,1358917,0,4227,2518,3,81919,1,0,2,1,8,0,0,5.610,2050
1364067,1364068,0,4227,2518,5,190835,1,0,1,4,8,0,0,18.248,2050
1364628,1364629,0,4227,2518,3,64330,3,0,5,1,8,0,0,2.338,2050
1369722,1369723,0,4227,2518,3,97732,1,0,5,3,8,0,0,3.756,2050


# SQL Connection 
Using both persons and households ds

In [169]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=DDAMWSQL16.sandag.org;'
                    'Database=isam;'
                    'Trusted_Connection=yes;')

query_all = """SELECT * 
FROM  [ddamwsql16].[isam].[xpef41].[abm_mgra13_based_input_np]"""


sql_query = pd.read_sql_query(query_all,conn)
sql_query

,yr,mgra,taz,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,...,luz_id,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,MicroAccessTime,remoteAVParking,refueling_stations
0,2029,3597,3529,20,0,20,0,15,0,15,...,99,1,26,0.33,3.64,3.64,3.64,3,0,1
1,2030,3597,3529,23,0,23,0,17,0,17,...,99,1,26,0.33,3.64,3.64,3.64,3,0,1
2,2032,3597,3529,29,0,29,0,20,0,20,...,99,1,26,0.33,3.64,3.64,3.64,3,0,1
3,2035,3597,3529,33,0,33,0,24,0,24,...,99,1,26,0.33,3.64,3.64,3.64,3,0,1
4,2040,3597,3529,40,0,40,0,27,0,27,...,99,1,26,0.33,3.64,3.64,3.64,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322023,2032,23002,1254,0,0,0,0,0,0,0,...,14,1,1,2.46,50.82,34.13,50.82,120,0,0
322024,2035,23002,1254,0,0,0,0,0,0,0,...,14,1,1,2.46,50.82,34.13,50.82,120,0,0
322025,2040,23002,1254,0,0,0,0,0,0,0,...,14,1,1,2.46,50.82,34.13,50.82,120,0,0
322026,2045,23002,1254,0,0,0,0,0,0,0,...,14,1,1,2.46,50.82,34.13,50.82,120,0,0


In [170]:
ds36R_mgra = pd.read_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Data\mgra_DS36R_ind_QA.csv')

In [171]:
sql_query_1 = sql_query[sql_query['yr'].isin(list(set(ds36R_mgra['year'])))]
sql_query_1 = sql_query_1.rename(columns={'yr':'year'})


sql_query_1

,year,mgra,taz,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,...,luz_id,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,MicroAccessTime,remoteAVParking,refueling_stations
0,2029,3597,3529,20,0,20,0,15,0,15,...,99,1,26,0.33,3.64,3.64,3.64,3,0,1
1,2030,3597,3529,23,0,23,0,17,0,17,...,99,1,26,0.33,3.64,3.64,3.64,3,0,1
2,2032,3597,3529,29,0,29,0,20,0,20,...,99,1,26,0.33,3.64,3.64,3.64,3,0,1
3,2035,3597,3529,33,0,33,0,24,0,24,...,99,1,26,0.33,3.64,3.64,3.64,3,0,1
4,2040,3597,3529,40,0,40,0,27,0,27,...,99,1,26,0.33,3.64,3.64,3.64,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322023,2032,23002,1254,0,0,0,0,0,0,0,...,14,1,1,2.46,50.82,34.13,50.82,120,0,0
322024,2035,23002,1254,0,0,0,0,0,0,0,...,14,1,1,2.46,50.82,34.13,50.82,120,0,0
322025,2040,23002,1254,0,0,0,0,0,0,0,...,14,1,1,2.46,50.82,34.13,50.82,120,0,0
322026,2045,23002,1254,0,0,0,0,0,0,0,...,14,1,1,2.46,50.82,34.13,50.82,120,0,0


In [172]:
sql_query_1[ds36R_mgra.columns].columns == ds36R_mgra.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [173]:
new_sql= sql_query_1[ds36R_mgra.columns]

In [174]:
new_sql = new_sql.groupby(['year','mgra']).sum().reset_index()

In [176]:
ds36R_mgra = ds36R_mgra.groupby(['year','mgra']).sum().reset_index()

In [177]:
import pandas as pd

def unshared_columns(df1, df2):
    # Get the list of column names for each dataframe
    df1_columns = set(df1.columns)
    df2_columns = set(df2.columns)
    
    # Find the columns that are not shared between the two dataframes
    unshared_columns = list(df1_columns.symmetric_difference(df2_columns))
    
    return unshared_columns


In [178]:
unshared_columns(ds36R_mgra, new_sql)

['empdenbin',
 'totintbin',
 'retempden',
 'dudenbin',
 'totint',
 'empden',
 'duden',
 'popden']

In [179]:
def create_diff_df(df1, df2, geo_level):

    # Adjust for common years
    common_years = common_values(df1['year'], df2['year'])
    df1 = df1[df1['year'].isin(common_years)]
    df2 = df2[df2['year'].isin(common_years)]

    # Adjust for common geography levels
    common_geographies = common_values(df1[geo_level], df2[geo_level])
    df1 = df1[df1[geo_level].isin(common_geographies)]
    df2 = df2[df2[geo_level].isin(common_geographies)]

    # Remove other geography related columns
    # It doesn't matter which df goes inside
    wanted_columns = wanted_geography_cols(df1, geo_level)
    df1 = df1[wanted_columns]
    df2 = df2[wanted_columns]

    # Common Columns
    common_columns = common_values(df1.columns, df2.columns)
    df1 = df1[common_columns]
    df2 = df2[common_columns]

    # Set necessary index
    df1 = df1.set_index([geo_level, 'year'])
    df2 = df2.set_index([geo_level, 'year'])

    # Drop non-numeric columns
    df1 = df1.select_dtypes(include='number')
    df2 = df2.select_dtypes(include='number')

    # Ensure columns are identical
    assert sum(~(df1.columns == df2.columns)) == 0

    # Create the diff file
    output = df1.subtract(df2)

    # Order the columns according to original files - It doesn't matter which DSID is inputted
    correctly_ordered_columns_df = output.reset_index()[common_columns_names_in_order(
        output.reset_index(), '36R', geo_level)]

    correctly_ordered_columns_df = correctly_ordered_columns_df.set_index([
                                                                          geo_level, 'year'])

    return correctly_ordered_columns_df

In [180]:
diff_output = create_diff_df(new_sql, ds36R_mgra[new_sql.columns], geo_level='mgra')
diff_output

,,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,gq_civ,gq_mil,...,hotelroomtotal,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,MicroAccessTime,remoteAVParking,refueling_stations
mgra,year,,,,,,,,,,,,,,,,,,,,,
1,2016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0003,0.004556,-0.001482,0.004556,0,0,0
2,2016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0039,0.000815,0.000815,0.000815,0,0,0
3,2016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.0039,0.004876,0.002062,0.004876,0,0,0
4,2016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0018,0.003822,0.003822,0.003822,0,0,0
5,2016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0038,-0.002502,-0.003693,-0.002502,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22998,2050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0007,-0.001522,-0.003578,-0.001522,0,0,0
22999,2050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.0003,-0.002780,0.004905,-0.002780,0,0,0
23000,2050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.0021,0.004725,0.000184,0.004725,0,0,0


In [181]:
diff_output

,,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,gq_civ,gq_mil,...,hotelroomtotal,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,MicroAccessTime,remoteAVParking,refueling_stations
mgra,year,,,,,,,,,,,,,,,,,,,,,
1,2016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0003,0.004556,-0.001482,0.004556,0,0,0
2,2016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0039,0.000815,0.000815,0.000815,0,0,0
3,2016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.0039,0.004876,0.002062,0.004876,0,0,0
4,2016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0018,0.003822,0.003822,0.003822,0,0,0
5,2016,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0038,-0.002502,-0.003693,-0.002502,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22998,2050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.0007,-0.001522,-0.003578,-0.001522,0,0,0
22999,2050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.0003,-0.002780,0.004905,-0.002780,0,0,0
23000,2050,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.0021,0.004725,0.000184,0.004725,0,0,0


In [182]:
diff_output.to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Results\SQL_input_file_diff.csv')

In [184]:
import pandas as pd

def summary_stats(df):
    # Create an empty DataFrame to store the summary statistics
    summary_df = pd.DataFrame(columns=['min', 'max', 'mean', 'median'])
    
    # Iterate through each column in the input DataFrame
    for col in df.columns:
        # Calculate the minimum value, maximum value, mean, and median for the current column
        col_min = df[col].min()
        col_max = df[col].max()
        col_mean = df[col].mean()
        col_median = df[col].median()
        
        # Add the calculated statistics to the summary DataFrame
        summary_df.loc[col] = [col_min, col_max, col_mean, col_median]
    
    return summary_df


In [185]:
diff_output[['hstallsoth','hstallssam','hparkcost','dstallsoth','dstallssam','dparkcost','mstallsoth','mstallssam','mparkcost','parkactive','openspaceparkpreserve']]

,,hstallsoth,hstallssam,hparkcost,dstallsoth,dstallssam,dparkcost,mstallsoth,mstallssam,mparkcost,parkactive,openspaceparkpreserve
mgra,year,,,,,,,,,,,
1,2016,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0048
2,2016,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0050
3,2016,0,0,0.0,0,0,0.0,0,0,0.0,0.0,-0.0005
4,2016,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0021
5,2016,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0027
...,...,...,...,...,...,...,...,...,...,...,...,...
22998,2050,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0004
22999,2050,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0000
23000,2050,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0041


In [186]:
summary_stats(diff_output[['hstallsoth','hstallssam','hparkcost','dstallsoth','dstallssam','dparkcost','mstallsoth','mstallssam','mparkcost','parkactive','openspaceparkpreserve']])

,min,max,mean,median
hstallsoth,0.000000,0.000,0.000000,0.0
hstallssam,0.000000,0.000,0.000000,0.0
hparkcost,-0.867285,0.000,-0.118285,0.0
dstallsoth,-1156.000000,1060.000,0.019323,0.0
dstallssam,-1156.000000,1059.000,0.019366,0.0
dparkcost,-0.979428,0.000,-0.097213,0.0
mstallsoth,0.000000,0.000,0.000000,0.0
mstallssam,0.000000,0.000,0.000000,0.0
mparkcost,-0.905681,0.000,-0.089600,0.0
parkactive,-0.004900,0.005,0.000003,0.0


In [188]:
final = pd.DataFrame(diff_output.sum())
final.columns = ['total_diff']

final[final['total_diff'] != 0].to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Results\SQL_input_file_diff_nonzero.csv')

In [57]:
final

,total_diff
hs,0.000000e+00
hs_sf,0.000000e+00
hs_mf,0.000000e+00
hs_mh,0.000000e+00
hh,0.000000e+00
...,...
effective_acres,8.815444e-01
land_acres,7.853186e+00
MicroAccessTime,6.091214e+06
remoteAVParking,0.000000e+00


In [51]:
def create_diff_df(df1, df2, geo_level):

    # Adjust for common years
    common_years = common_values(df1['year'], df2['year'])
    df1 = df1[df1['year'].isin(common_years)]
    df2 = df2[df2['year'].isin(common_years)]

    # Adjust for common geography levels
    common_geographies = common_values(df1[geo_level], df2[geo_level])
    df1 = df1[df1[geo_level].isin(common_geographies)]
    df2 = df2[df2[geo_level].isin(common_geographies)]

    # Remove other geography related columns
    # It doesn't matter which df goes inside
    wanted_columns = wanted_geography_cols(df1, geo_level)
    df1 = df1[wanted_columns]
    df2 = df2[wanted_columns]

    # Common Columns
    common_columns = common_values(df1.columns, df2.columns)
    df1 = df1[common_columns]
    df2 = df2[common_columns]

    # Set necessary index
    df1 = df1.set_index([geo_level, 'year'])
    df2 = df2.set_index([geo_level, 'year'])

    # Drop non-numeric columns
    df1 = df1.select_dtypes(include='number')
    df2 = df2.select_dtypes(include='number')

    # Ensure columns are identical
    assert sum(~(df1.columns == df2.columns)) == 0

    # Create the diff file
    output = df1.subtract(df2)

    # Order the columns according to original files - It doesn't matter which DSID is inputted
    correctly_ordered_columns_df = output.reset_index()[common_columns_names_in_order(
        output.reset_index(), '36R', geo_level)]

    correctly_ordered_columns_df = correctly_ordered_columns_df.set_index([
                                                                          geo_level, 'year'])

    return correctly_ordered_columns_df

In [ ]:
# Get the raw data from DDAM
query = """
SELECT * 
FROM [ddamwsql16].[isam].[xpef41].[abm_mgra13_based_input]
"""

# Get the table

table = pd.read_sql_query(query, con=DDAM).drop("county_fips_code", axis=1)

# Order the table by "fiscal_yr" to make viewing slices more enjoyable
table = table.sort_values("fiscal_yr").reset_index(drop=True)

# download files

In [4]:
ds36R_mgra = pd.read_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Data\mgra_DS36R_ind_QA.csv')


In [29]:
persons_households_pop_comp_gq_only = pd.read_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Data\DS36R_persons_household_population_comparison_GQ_only_QA.csv')

In [30]:
persons_households_pop_comp_gq_only

,hhid,Persons_Dataset_Pop,Households_Dataset_Pop,Diff_P_minus_H,year
0,886,1,NaN,NaN,2016
1,2000,1,NaN,NaN,2016
2,2554,1,NaN,NaN,2016
3,2608,1,NaN,NaN,2016
4,2852,1,NaN,NaN,2016
...,...,...,...,...,...
1066005,1453334,1,1.0,0.0,2050
1066006,1453335,1,1.0,0.0,2050
1066007,1453336,1,1.0,0.0,2050
1066008,1453337,1,1.0,0.0,2050


In [31]:
persons_households_pop_comp_all[persons_households_pop_comp_all['Diff_P_minus_H'] != 0]

,hhid,Persons_Dataset_Pop,Households_Dataset_Pop,Diff_P_minus_H,year


In [22]:
households_input_files_pop_comp_gq_only[households_input_files_pop_comp_gq_only['Diff'] != 0].to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Results\households_and_input_files_pop_comp_nonzero_diff_gq_only.csv')

In [18]:
households_input_files_pop_comp_all[households_input_files_pop_comp_all['Diff'] != 0].to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Results\households_and_input_files_pop_comp_nonzero_diff.csv')

In [12]:
na_check = pd.DataFrame(ds36R_mgra.isna().sum())
na_check.columns = ['Number of NaN']
na_check[na_check['Number of NaN'] != 0]
# .to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Results\NaN_values.csv')

,Number of NaN
totint,299026
duden,299026
empden,299026
popden,299026
retempden,299026
totintbin,299026
empdenbin,299026
dudenbin,299026


In [2]:
region_36_ind = region_foldup(dsid='36', to_jdrive=False)
region_36R_ind = region_foldup(dsid='36R', to_jdrive=False)

In [5]:
region_36_ind = download_ind_file('36', geo_level='region')
region_36R_ind = download_ind_file('36R', geo_level='region')

In [10]:
mgra_36_ind = download_ind_file('36', geo_level='mgra')

In [11]:
mgra_36R_ind = download_ind_file('36R', geo_level='mgra')

In [15]:
sum(mgra_36_ind[mgra_36_ind['year'] == 2032]['emp_total']) - sum(mgra_36R_ind[mgra_36R_ind['year'] == 2032]['emp_total'])

1902

In [27]:
region_36_ind[emp_cols]

,emp_ag,emp_amusement,emp_const_bldg_office,emp_const_bldg_prod,emp_const_non_bldg_office,emp_const_non_bldg_prod,emp_fed_mil,emp_fed_non_mil,emp_health,emp_hotel,...,emp_pvt_hh,emp_restaurant_bar,emp_retail,emp_state_local_gov_blue,emp_state_local_gov_ent,emp_state_local_gov_white,emp_trans,emp_utilities_office,emp_utilities_prod,emp_whsle_whs
0,9300,45639,18484,53998,2853,8163,126948,11454,199700,33094,...,16471,131594,156612,9337,18797,16332,36183,4479,2729,46625
1,9616,47191,19897,58116,2129,6334,127708,11841,206625,34216,...,17137,136111,161823,9644,19408,16904,37415,4693,2759,48228
2,9617,47201,19931,58217,2132,6348,131042,11841,206789,34218,...,17398,136169,161823,9644,19411,16901,37423,4695,2759,48252
3,9804,48078,20526,59778,2099,6193,135461,12057,210810,34830,...,18119,138774,164666,9811,19752,17205,38133,4810,2797,49183
4,9907,48626,20767,60664,2069,6108,138402,12195,213387,35246,...,18296,140398,166423,9911,19972,17407,38559,4873,2805,49758
5,9987,49066,20946,61243,2071,6126,138629,12309,215455,35574,...,18368,141705,167840,9996,20151,17564,38912,4895,2840,50219
6,10300,50511,21822,63256,2136,6284,139309,12657,221888,36593,...,19212,145923,172661,10291,20710,18068,40057,5013,2975,51727
7,10380,50951,22001,63835,2138,6302,139536,12771,223956,36921,...,19284,147230,174078,10376,20889,18225,40410,5035,3010,52188
8,10550,51863,22418,65058,2130,6347,139987,12997,228141,37581,...,19497,149921,177003,10567,21247,18537,41135,5146,3028,53131
9,10851,53264,23276,67037,2157,6482,140662,13346,234502,38587,...,20199,154036,181656,10867,21792,19013,42248,5324,3086,54607


In [33]:
emp_cols = ['emp_ag',
'emp_amusement',
'emp_const_bldg_office',
'emp_const_bldg_prod',
'emp_const_non_bldg_office',
'emp_const_non_bldg_prod',
'emp_fed_mil',
'emp_fed_non_mil',
'emp_health',
'emp_hotel',
'emp_mfg_office',
'emp_mfg_prod',
'emp_personal_svcs_office',
'emp_personal_svcs_retail',
'emp_prof_bus_svcs',
'emp_prof_bus_svcs_bldg_maint',
'emp_public_ed',
'emp_pvt_ed_k12',
'emp_pvt_ed_post_k12_oth',
'emp_pvt_hh',
'emp_restaurant_bar',
'emp_retail',
'emp_state_local_gov_blue',
'emp_state_local_gov_ent',
'emp_state_local_gov_white',
'emp_trans',
'emp_utilities_office',
'emp_utilities_prod',
'emp_whsle_whs']

In [ ]:
output = mgra_36R_ind[['mgra', 'year']]
output['emp_total_diff_agg_minus_emptotal'] = mgra_36R_ind[emp_cols].sum(axis=1) - mgra_36R_ind['emp_total']
output

In [46]:
output = mgra_36R_ind[['mgra', 'year']]
output['emp_total_diff_agg_minus_emptotal'] = mgra_36R_ind[emp_cols].sum(axis=1) - mgra_36R_ind['emp_total']
output

,mgra,year,emp_total_diff_agg_minus_emptotal
0,1,2016,0
1,2,2016,0
2,3,2016,0
3,4,2016,0
4,5,2016,0
...,...,...,...
299021,22998,2050,0
299022,22999,2050,0
299023,23000,2050,0
299024,23001,2050,0


In [47]:
output[output['emp_total_diff_agg_minus_emptotal'] != 0]

,mgra,year,emp_total_diff_agg_minus_emptotal


In [49]:
# Have this check at an MGRA level as well ---- 
output = region_36R_ind[['region', 'year']]
output['emp_total_diff_agg_minus_emptotal'] = region_36R_ind[emp_cols].sum(axis=1) - region_36R_ind['emp_total']
output

,region,year,emp_total_diff_agg_minus_emptotal
0,San Diego,2016,0
1,San Diego,2018,0
2,San Diego,2020,0
3,San Diego,2023,0
4,San Diego,2025,0
5,San Diego,2026,0
6,San Diego,2029,0
7,San Diego,2030,0
8,San Diego,2032,0
9,San Diego,2035,0


In [48]:
# Have this check at an MGRA level as well ---- 
output = region_36_ind[['region', 'year']]
output['emp_total_diff_agg_minus_emptotal'] = region_36_ind[emp_cols].sum(axis=1) - region_36_ind['emp_total']
output
# output.to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Data\emp_total_diff_agg_minus_emptotal.csv')

,region,year,emp_total_diff_agg_minus_emptotal
0,San Diego,2016,0
1,San Diego,2018,0
2,San Diego,2020,0
3,San Diego,2023,1022
4,San Diego,2025,0
5,San Diego,2026,-939
6,San Diego,2029,939
7,San Diego,2030,0
8,San Diego,2032,-1016
9,San Diego,2035,0


In [ ]:
# Have this check at an MGRA level as well ---- 
output = mgra_36R_ind[['region', 'year']]
output['emp_total_diff_agg_minus_emptotal'] = region_36_ind[emp_cols].sum(axis=1) - region_36_ind['emp_total']
output.to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Data\emp_total_diff_agg_minus_emptotal.csv')

In [38]:
region_36_ind

,region,year,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,...,hotelroomtotal,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,MicroAccessTime,remoteAVParking,refueling_stations
0,San Diego,2016,1190554,717626,430716,42212,1134848,687509,407158,40181,...,56646,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
1,San Diego,2018,1205852,722804,440941,42107,1147635,692120,415481,40034,...,61917,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
2,San Diego,2020,1226462,726240,458115,42107,1166240,696042,430088,40110,...,61917,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2333475,5,699
3,San Diego,2023,1262957,728452,492398,42107,1197072,698421,458511,40140,...,63707,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,2049556,5,699
4,San Diego,2025,1288217,729790,516320,42107,1219745,699768,479824,40153,...,64889,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,1688220,5,699
5,San Diego,2026,1300847,730445,528295,42107,1231007,700393,490458,40156,...,65379,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,1688220,5,699
6,San Diego,2029,1338737,732381,564249,42107,1264151,702272,521720,40159,...,66938,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,1688220,5,699
7,San Diego,2030,1351367,732915,576345,42107,1274948,702824,531964,40160,...,67428,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,1688220,5,699
8,San Diego,2032,1376162,734194,599861,42107,1296193,703940,552092,40161,...,68233,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,1688220,5,699
9,San Diego,2035,1409867,735001,632759,42107,1327588,705060,582365,40163,...,69454,23002,221100,245307.3086,2.727204e+06,1.251249e+06,2.698589e+06,1688220,5,699


In [42]:
# Have this check at an MGRA level as well ---- 
output = region_36_ind[['region', 'year']]
output['emp_total_diff_agg_minus_emptotal'] = region_36_ind[emp_cols].sum(axis=1) - region_36_ind['emp_total']
output.to_csv(r'C:\Users\cra\San Diego Association of Governments\SANDAG QA QC - Documents\Projects\2023\2023-002 XPEF41 (DS 36 Revised) Forecast\Data\emp_total_diff_agg_minus_emptotal.csv')

In [37]:
region_36_ind[emp_cols].sum(axis=1) - region_36_ind['emp_total']

0        0
1        0
2        0
3     1022
4        0
5     -939
6      939
7        0
8    -1016
9        0
10       0
11       0
12       0
dtype: int64

In [29]:
region_36_ind['emp_total']

0     1646419
1     1699710
2     1704871
3     1739844
4     1762720
5     1778595
6     1827368
7     1843243
8     1874712
9     1922471
10    1988442
11    2045618
12    2087328
Name: emp_total, dtype: int64

In [24]:
emp_cols = ['emp_ag',
'emp_amusement',
'emp_const_bldg_office',
'emp_const_bldg_prod',
'emp_const_non_bldg_office',
'emp_const_non_bldg_prod',
'emp_fed_mil',
'emp_fed_non_mil',
'emp_health',
'emp_hotel',
'emp_mfg_office',
'emp_mfg_prod',
'emp_personal_svcs_office',
'emp_personal_svcs_retail',
'emp_prof_bus_svcs',
'emp_prof_bus_svcs_bldg_maint',
'emp_public_ed',
'emp_pvt_ed_k12',
'emp_pvt_ed_post_k12_oth',
'emp_pvt_hh',
'emp_restaurant_bar',
'emp_retail',
'emp_state_local_gov_blue',
'emp_state_local_gov_ent',
'emp_state_local_gov_white',
'emp_trans',
'emp_utilities_office',
'emp_utilities_prod',
'emp_whsle_whs']

In [16]:
t_drive_36_input_2032 = pd.read_csv('T:/socioec/Current_Projects/XPEF32/abm_csv/new_parking/mgra13_based_input2032_05_np.csv')

In [19]:
t_drive_36R_input_2032 = pd.read_csv('T:/socioec/Current_Projects/XPEF41/abm_csv/new_parking/mgra13_based_input2032_05_np.csv')

In [23]:
sum(t_drive_36_input_2032['emp_total']) - sum(t_drive_36R_input_2032['emp_total'])

1902

In [22]:
sum(t_drive_36R_input_2032['emp_total'])

1872810

In [6]:
my_region_output = pd.read_csv(r'J:\DataScience\DataQuality\QAQC\forecast_automation\mgra_series_13_outputs_CSV_data\diff_files\region_diff_DS36_minus_DS36R_QA.csv')

In [9]:
region_36_ind['emp_total'] - region_36R_ind['emp_total']

0        8
1      510
2      567
3     -685
4      240
5     1275
6     -232
7      803
8     1902
9      979
10    1012
11    1017
12    1013
Name: emp_total, dtype: int64

In [8]:
my_region_output['emp_total']

0        8
1      510
2      567
3     -685
4      240
5     1275
6     -232
7      803
8     1902
9      979
10    1012
11    1017
12    1013
Name: emp_total, dtype: int64

In [ ]:
my_region_output = pd.read_csv(r'J:\DataScience\DataQuality\QAQC\forecast_automation\mgra_series_13_outputs_CSV_data\diff_files\region_diff_DS36_minus_DS36R_QA.csv')